In [21]:
from datetime import date
import datetime
import unittest
import ipynb.fs.full.M3_B2_PROJECT_3 as pj3
import pandas as pd
from tabulate import tabulate 

In [22]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity='all'

In [23]:
# Story US01 Task T01.03
#Check the Individual table for Birth dates and Death dates 

def story_us01(indiTable,famTable):
    for row in indiTable:
        row.border=False
        row.header=False
        row_birthdate = (row.get_string(fields = ["Birth Date"]).strip())
        row_deathdate = (row.get_string(fields = ["Death Date"]).strip())
        if isDateInPast(row_birthdate) is False:
            print("Error:","Individual ID ",row.get_string(fields = ["Birth Date"]).strip(), "has Date of Birth in Future" )
        else: print("Record Check OK for Birth Dates ")
# Story US01 Task T01.04         
        if row_deathdate == 'NA':
            print("Record Check OK for Death Dates ")
            continue
        elif isDateInPast(row_deathdate) is False:
            print("Error:","Individual ID ",row.get_string(fields = ["Birth Date"]), "has Date of Death in Future" ) 
        else: print("Record Check OK for Death Dates ")
# Story US01 Task T01.05
# Check Marriage Dates in Family table 
    for row in famTable:
        row.border=False
        row.header=False
        row_marriagedate = (row.get_string(fields = ["Married"]).strip())
        if isDateInPast(row_marriagedate) is False:
            print("Error:","Family ID",row.get_string(fields = ["ID"]).strip()
                  ,row.get_string(fields = ["Marriage Date"]), "has Date of Wedding in Future" )
        else: print("Record:","Family ID",row.get_string(fields = ["ID"]).strip(),"Check OK for Wedding Dates ")

In [24]:
# Story US02 
# Birth before Marriage
def story_us02(indiTable,famTable):
# Task-T02.02 List out Individuals who are married 
    df_indi = pd.DataFrame()
    df_fam = pd.DataFrame()
    df_indi,df_fam = dataframe(indiTable,famTable)
    df_husband = df_fam.loc[:,['ID','Married','Husband ID',"Husband's Name"]]
    # Task-T02.03 List out dates for marriage for individuals 
    df_husband = df_husband.merge(df_indi.loc[:,['ID','Birth Date']],
                                  left_on='Husband ID',right_on='ID',
                                  how='inner',suffixes=('_Family','_Individual'))
    # Task-T02.04 Compare birth and death rates 
    df_husband['Date Check'] = df_husband.apply(lambda row : compareDate(row['Married'],row['Birth Date']),axis = 1)
    
    df_wife = df_fam.loc[:,['ID','Married','Wife ID',"Wife's Name"]]
    df_wife = df_wife.merge(df_indi.loc[:,['ID','Birth Date']],
                                  left_on='Wife ID',right_on='ID',
                                  how='inner',suffixes=('_Family','_Individual'))
    # Task-T02.04 Compare birth and death rates 
    df_wife['Date Check'] = df_wife.apply(lambda row : compareDate(row['Married'],row['Birth Date']),axis = 1)
    
    print('Records OK for Marriage Dates')
    df_result_husband = df_husband[df_husband['Date Check']==True]
    print(tabulate(df_result_husband.loc[:,['ID_Family','Married','Husband ID',"Husband's Name", 'Birth Date']],
                   headers = 'keys', tablefmt = 'psql'))
    
    df_result_wife = df_wife[df_wife['Date Check']==True]
    print(tabulate(df_result_wife.loc[:,['ID_Family','Married','Wife ID',"Wife's Name", 'Birth Date']],
                   headers = 'keys', tablefmt = 'psql'))
    
    print("Records Error for Marriage Dates ")
    df_result_husband_e = df_husband[df_husband['Date Check']==False]
    print(tabulate(df_result_husband_e.loc[:,['ID_Family','Married','Husband ID',"Husband's Name", 'Birth Date']], 
                   headers = 'keys', tablefmt = 'psql'))
    df_result_wife_e = df_wife[df_wife['Date Check']==False]
    print(tabulate(df_result_wife_e.loc[:,['ID_Family','Married','Wife ID',"Wife's Name", 'Birth Date']],
                   headers = 'keys', tablefmt = 'psql'))

In [25]:
def dataframe(indiTable,famTable):
    df_indi = pd.DataFrame(columns=indiTable.field_names)
    df_fam = pd.DataFrame(columns=famTable.field_names)
    row_list = []
    for row in indiTable:
        row.border=False
        row.header=False
        row_ID = (row.get_string(fields = ["ID"]).strip())
        row_name = (row.get_string(fields = ["Name"]).strip())
        row_sex = (row.get_string(fields = ["Sex"]).strip())
        row_birthdate = (row.get_string(fields = ["Birth Date"]).strip())
        row_age = (row.get_string(fields = ["Age"]).strip())
        row_alive = (row.get_string(fields = ["Alive"]).strip())
        row_deathdate = (row.get_string(fields = ["Death Date"]).strip())
        row_spouse = (row.get_string(fields = ["Spouse"]).strip())
        row_child = (row.get_string(fields = ["Child"]).strip())
        row_list = {'ID':row_ID,'Name':row_name,
                             'Sex':row_sex,'Birth Date':row_birthdate,
                             'Age':row_age,'Alive':row_alive,
                             'Death Date':row_deathdate,'Spouse':row_spouse,
                             'Child':row_child}
        df_indi = df_indi.append(row_list,ignore_index=True)

    row_list=[]   
    for row in famTable:
        row.border=False
        row.header=False
        row_ID = (row.get_string(fields = ["ID"]).strip())
        row_married = (row.get_string(fields = ["Married"]).strip())
        row_divorced = (row.get_string(fields = ["Divorced"]).strip())
        row_husbandID = (row.get_string(fields = ["Husband ID"]).strip())
        row_husband = (row.get_string(fields = ["Husband's Name"]).strip())
        row_wifeID = (row.get_string(fields = ["Wife ID"]).strip())
        row_wife = (row.get_string(fields = ["Wife's Name"]).strip())
        row_children = (row.get_string(fields = ["Children"]).strip())
        row_list = {'ID':row_ID,'Married':row_married,
                    'Divorced':row_divorced,'Husband ID':row_husbandID,
                    "Husband's Name":row_husband ,"Wife ID":row_wifeID,
                    "Wife's Name":row_wife,'Children':row_children}
        df_fam = df_fam.append(row_list,ignore_index=True)
    return df_indi, df_fam

In [26]:
def splitDate(dateStr):
    dateArr = dateStr.split(" ")
    return dateArr

In [27]:
def getMonth(monthStr):
    full_date = datetime.datetime.strptime(monthStr, "%b")
    return full_date.month

In [28]:
# Story US01 Task T01.02
### function to check if the passed date is in the past or not
### returns true if date is in the past
def isDateInPast(eachDate):
    today = str(date.today())
    today = today.split("-")
    compareDate = splitDate(eachDate)
    currentDate = datetime.date(int(today[0]), int(today[1]), int(today[2]))
    checkingDate = datetime.date(int(compareDate[0]), int(getMonth(compareDate[1])), int(compareDate[2]))
    
    compare = checkingDate < currentDate
    if (compare == False):
        return False; #returns False when  earlier date is more than later date
    else:
        return True

In [29]:
# Function to compare Dates 
#Enter later date first(date1) and Earlier date second(date2)
def compareDate(date1,date2):
    date_1 = splitDate(date1)
    date_2 = splitDate(date2)
    comparedate1 = datetime.date(int(date_1[0]), int(getMonth(date_1[1])), int(date_1[2]))
    comparedate2 = datetime.date(int(date_2[0]), int(getMonth(date_2[1])), int(date_2[2]))
    compare = comparedate2 < comparedate1
    if (compare == False):
        return False;
    else:
        return True

In [30]:
### unit tests to check if the passed date is in the past or not
class TestDateInPast(unittest.TestCase):
    ## test if past date gives True
    def test_past_date(self):
        self.assertEqual(isDateInPast("2019 JAN 15"), True)
    ## test if future date gives False
    def test_future_date(self):
        self.assertEqual(isDateInPast("2025 MAR 26"), False)
    ## test if past date gives True with assertTrue
    def test_past_true(self):
        self.assertTrue(isDateInPast("1958 Feb 4"))
    ## test if future date gives False with assertFalse
    def test_future_false(self):
        self.assertFalse(isDateInPast("2022 Dec 12"))
    ## test if current date gives False with assertTrue
    def test_current(self):
        self.assertEqual(isDateInPast("2022 Jun 6"), False)

In [31]:
unittest.main(argv=[''], verbosity=2, exit=False)

test_current (__main__.TestDateInPast) ... FAIL
test_future_date (__main__.TestDateInPast) ... ok
test_future_false (__main__.TestDateInPast) ... ok
test_past_date (__main__.TestDateInPast) ... ok
test_past_true (__main__.TestDateInPast) ... ok

FAIL: test_current (__main__.TestDateInPast)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "C:\Users\sidha\AppData\Local\Temp\ipykernel_3792\1010258619.py", line 17, in test_current
    self.assertEqual(isDateInPast("2022 Jun 6"), False)
AssertionError: True != False

----------------------------------------------------------------------
Ran 5 tests in 0.004s

FAILED (failures=1)


In [32]:
file_path = r"C:\Users\sidha\Stevens\CS 555 - Agile Methods for Software Development\Project\Project 3\Family tree-Sprint1-UAT.ged"
indiList,famList = pj3.gedcomRead(file_path)
indiTable , famTable = pj3.prettytab(indiList,famList)

Individials
Families


In [33]:
print('Individuals')
pj3.printtable(indiTable)
print('Families')
pj3.printtable(famTable)

Individuals
+-------+----------------+-----+-------------+-----+-------+-------------+--------+-------+
|   ID  |      Name      | Sex |  Birth Date | Age | Alive |  Death Date | Spouse | Child |
+-------+----------------+-----+-------------+-----+-------+-------------+--------+-------+
| @I10@ | Shivkumar Rao  |  M  | 1995 SEP 13 |  26 |  True |      NA     |  @F8@  |   NA  |
| @I11@ |  Keshav Kumar  |  M  |  1985 JAN 2 |  37 |  True |      NA     |  @F9@  |  @F5@ |
| @I12@ | Priya Pradhan  |  F  | 1965 JUN 19 |  56 |  True |      NA     |  @F8@  |   NA  |
| @I13@ |   Shilpa Rao   |  F  | 1983 AUG 21 |  38 |  True |      NA     | @F10@  |  @F8@ |
| @I14@ |   Satish Rao   |  M  | 1994 JUN 18 |  27 |  True |      NA     |   NA   |  @F4@ |
| @I15@ |   Arun Gupta   |  M  |  1982 OCT 5 |  39 |  True |      NA     |  @F7@  |   NA  |
| @I16@ |  Nisha Gupta   |  F  | 2009 DEC 24 |  12 |  True |      NA     |   NA   |  @F7@ |
| @I17@ |   Nina Singh   |  F  | 1989 JAN 31 |  33 |  True |      NA

In [34]:
story_us01(indiTable, famTable)
story_us02(indiTable,famTable)

Record Check OK for Birth Dates 
Record Check OK for Death Dates 
Record Check OK for Birth Dates 
Record Check OK for Death Dates 
Record Check OK for Birth Dates 
Record Check OK for Death Dates 
Record Check OK for Birth Dates 
Record Check OK for Death Dates 
Record Check OK for Birth Dates 
Record Check OK for Death Dates 
Record Check OK for Birth Dates 
Record Check OK for Death Dates 
Record Check OK for Birth Dates 
Record Check OK for Death Dates 
Record Check OK for Birth Dates 
Record Check OK for Death Dates 
Record Check OK for Birth Dates 
Record Check OK for Death Dates 
Record Check OK for Birth Dates 
Record Check OK for Death Dates 
Record Check OK for Birth Dates 
Record Check OK for Death Dates 
Record Check OK for Birth Dates 
Record Check OK for Death Dates 
Record Check OK for Birth Dates 
Record Check OK for Death Dates 
Record Check OK for Birth Dates 
Record Check OK for Death Dates 
Record Check OK for Birth Dates 
Record Check OK for Death Dates 
Record Che